In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from scipy import stats

import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

In [2]:
a=pd.read_csv('../parsing/full_database.csv')

In [3]:
del a['len_name']

In [4]:
a.head(2)

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace
0,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195,Abächerli Walter,M,1952,Hinwil,47,"4:31.56,1","0:53.5,3","0:6.26,0"
1,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,42.195,Abbringh Ellen,F,1962,NL-Doorn 3941 EB,91,"5:55.9,4","2:12.11,6","0:8.25,0"


In [5]:
weather_db=pd.read_csv('../datasets/races-information-weather.csv')
del weather_db['Unnamed: 0']
del weather_db['Unnamed: 0.1']

In [6]:
weather_db.head(2)

,Date,Name,Place,URL,min_temp,max_temp,uv_index,weather_desc
0,sam. 27.03.1999,Männedörfler Waldlauf,Männedorf,http://services.datasport.com/1999/zkb/maennedorf,NaN,NaN,NaN,NaN
1,sam. 20.03.1999,Kerzerslauf,Kerzers,http://services.datasport.com/1999/lauf/kerzers,NaN,NaN,NaN,NaN


In [7]:
gr=a.groupby([a.Date,a.Race])

In [8]:
def merge_weather_info(x):
    race=x.iloc[0].Race
    date=x.iloc[0].Date
    date2=date[:15]
    weather=weather_db[(weather_db.Name==race) & (weather_db.Date==date2)].iloc[0]
    x['Date']=date2
    x['Place']=weather.Place
    x['MinTemp']=weather.min_temp
    x['MaxTemp']=weather.max_temp
    x['Weather']=weather.weather_desc
    x['RaceID']=weather.URL
    
    return x

In [9]:
full_df=gr.apply(merge_weather_info)

In [10]:
full_df=full_df.dropna()

In [11]:
full_df.shape

(1103009, 16)

In [12]:
full_df.head(5)

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Delay,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
165242,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Helmut,M,1962,Staldenried,165,"6:10.37,2","2:32.19,2","0:8.47,0",Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...
165243,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Julia,F,1950,Gwatt (Thun),5,"5:34.8,2","0:38.30,9","0:7.55,0",Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...
165244,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Medard,M,1942,Visp,2,"5:13.54,7","0:27.3,3","0:7.26,0",Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...
165245,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Achterberg Nico,M,1957,NL-Veenendaal,80,"5:38.35,0","2:3.9,3","0:8.1,0",Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...
165246,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Ackeret Peter,M,1957,Deitingen,11,"4:26.3,5","0:50.37,8","0:6.18,0",Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...


# Parse times and select only meaningful features

In [13]:
base=datetime.datetime.strptime('0:0.0,0','%H:%M.%S,%f')

In [14]:
full_df['Pace'] = (pd.to_datetime(full_df.Pace, format='%H:%M.%S,%f')-base)

In [15]:
full_df.shape

(1103009, 16)

In [19]:
full_df=full_df[full_df.Distance<45]

In [20]:
max(full_df.Distance)

42.416259459459461

In [21]:
full_df.shape

(1096394, 16)

In [23]:
full_df['Time'] = (pd.to_datetime(full_df.Time, format='%H:%M.%S,%f')-base)

In [25]:
del full_df['Rank']
del full_df['Delay']

In [26]:
full_df.head(3)

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
165242,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Helmut,M,1962,Staldenried,06:10:37.200000,00:08:47,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...
165243,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Julia,F,1950,Gwatt (Thun),05:34:08.200000,00:07:55,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...
165244,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Medard,M,1942,Visp,05:13:54.700000,00:07:26,Zermatt,8.0,20.0,Partly cloudy,http://services.datasport.com/2008/lauf/zermat...


# Categorize better weather

In [41]:
full_df.Weather.value_counts().index

Index(['Partly cloudy', 'Clear', 'Sunny', 'Mist', 'Patchy rain possible',
       'Cloudy', 'Light rain', 'Light rain shower', 'Moderate rain',
       'Overcast', 'Fog', 'Moderate or heavy rain shower',
       'Moderate rain at times', 'Light drizzle', 'Light snow',
       'Patchy light drizzle', 'Moderate snow', 'Patchy light rain',
       'Patchy light rain with thunder', 'Freezing fog', 'Heavy snow',
       'Heavy rain', 'Moderate or heavy rain with thunder',
       'Moderate or heavy sleet', 'Patchy moderate snow',
       'Heavy rain at times', 'Light sleet',
       'Moderate or heavy snow with thunder'],
      dtype='object')

1 -> good weather
0 -> bad weather

In [42]:
dict_favorable_weather={'Partly cloudy':1, 'Clear':1, 'Sunny':1, 'Mist':0, 'Patchy rain possible':1,
       'Cloudy':1, 'Light rain':0, 'Light rain shower':0, 'Moderate rain':0,
       'Overcast':1, 'Fog':0, 'Moderate or heavy rain shower':0,
       'Moderate rain at times':1, 'Light drizzle':0, 'Light snow':0,
       'Patchy light drizzle':0, 'Moderate snow':0, 'Patchy light rain':0,
       'Patchy light rain with thunder':0, 'Freezing fog':0, 'Heavy snow':0,
       'Heavy rain':0, 'Moderate or heavy rain with thunder':0,
       'Moderate or heavy sleet':0, 'Patchy moderate snow':0,
       'Heavy rain at times':0, 'Light sleet':0,
       'Moderate or heavy snow with thunder':0}

In [44]:
def categorize_weather(x):
    return dict_favorable_weather[x]

In [46]:
full_df.Weather=full_df.Weather.apply(categorize_weather)

# Analysis of men

In [48]:
men_full_df=full_df[full_df.Sex=='M']

In [49]:
men_marathon=men_full_df[men_full_df.Distance==42.195]

In [50]:
men_marathon.shape

(53604, 14)

In [51]:
men_marathon.head(3)

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
165242,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Helmut,M,1962,Staldenried,06:10:37.200000,00:08:47,Zermatt,8.0,20.0,1,http://services.datasport.com/2008/lauf/zermat...
165244,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Abgottspon Medard,M,1942,Visp,05:13:54.700000,00:07:26,Zermatt,8.0,20.0,1,http://services.datasport.com/2008/lauf/zermat...
165245,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,Achterberg Nico,M,1957,NL-Veenendaal,05:38:35,00:08:01,Zermatt,8.0,20.0,1,http://services.datasport.com/2008/lauf/zermat...


In [78]:
def add_mean_std_race(x):
    x['MeanRace']=np.mean(x.Time)
    x['StdRace']=np.std(x.Time)/np.sqrt(x.shape[0])
    return x

In [79]:
men_marathon=men_marathon.groupby(men_marathon.RaceID).apply(add_mean_std_race)

In [80]:
men_marathon_means=men_marathon.groupby(men_marathon.RaceID).first()

In [81]:
men_marathon_means.head(1)

,Race,Date,Distance,Name,Sex,Year,LivingPlace,Time,Pace,Place,MinTemp,MaxTemp,Weather,MeanRace,StdRace
RaceID,,,,,,,,,,,,,,,
http://services.datasport.com/2008/lauf/jungfrau,"Jungfrau-Marathon, Interlaken",sam. 06.09.2008,42.195,Abächerli Walter,M,1952,Hinwil,04:57:05.200000,00:07:02,Interlaken,12.0,16.0,1,05:14:00.868006,00:00:42.743138


In [82]:
men_marathon_means=men_marathon_means[['Race','Date','Distance','MinTemp','MaxTemp','Weather','MeanRace','StdRace']]

In [83]:
men_marathon_means['AvgTemp']=(men_marathon_means.MinTemp+men_marathon_means.MaxTemp)/2

In [84]:
men_marathon_means=men_marathon_means[['Race','Date','Distance','Weather','AvgTemp','MeanRace','StdRace']]

In [85]:
men_marathon_means.head(5)

,Race,Date,Distance,Weather,AvgTemp,MeanRace,StdRace
RaceID,,,,,,,
http://services.datasport.com/2008/lauf/jungfrau,"Jungfrau-Marathon, Interlaken",sam. 06.09.2008,42.195,1,14.0,05:14:00.868006,00:00:42.743138
http://services.datasport.com/2008/lauf/tenero,"Maratona Ticino, Tenero",dim. 09.11.2008,42.195,1,4.5,03:41:43.842028,00:04:13.968175
http://services.datasport.com/2008/lauf/zermattmarathon,"Zermatt Marathon, Zermatt",sam. 05.07.2008,42.195,1,14.0,05:20:34.918863,00:01:33.758564
http://services.datasport.com/2008/waffenlauf/frauenfelder,Frauenfelder,dim. 16.11.2008,42.195,1,4.0,04:12:17.644525,00:02:37.133417
http://services.datasport.com/2009/lauf/biel,"Bieler Lauftage, Biel/Bienne",ven. 12.06.2009,42.195,1,16.0,04:24:31.801000,00:04:53.952692


In [ ]:
men_marathon_means.Weather.value_counts()

In [ ]:
gr_means=men_marathon_means.groupby(men_marathon_means.Race)

In [ ]:
for i,a in gr_means:
    print(a)